In [1]:
import requests
import time
import os

from zipfile import ZipFile

from io import BytesIO

In [4]:
def get_fars_data(start_year, end_year=None):

    base_url = "https://www.nhtsa.gov/file-downloads/download"

    if end_year is None:
        end_year = start_year
    
    for year in range(start_year, end_year + 1):

        path = f'./data/{year}_fars/'
        
        # try the request three times before giving up
        current_tries = 1
        while current_tries < 3:
            try:
                # get FARS .zip files
                res = requests.get(url=base_url, params={
                    'p': f"nhtsa/downloads/FARS/{year}/National/FARS{year}NationalCSV.zip"
                })
                assert res.status_code == 200  
                
                # extract FARS .zip files to folders by year
                with ZipFile(BytesIO(res.content)) as zfile:
                    zfile.extractall(path)
                
                break
            except:
                time.sleep(1)
                current_tries += 1
        
        # same for Auxiliary data
        current_tries = 1
        while current_tries < 3:
            try:
                res_aux = requests.get(url=base_url, params={
                    'p': f"nhtsa/downloads/FARS/{year}/National/FARS{year}NationalAuxiliaryCSV.zip"
                })
                assert res_aux.status_code == 200  
                with ZipFile(BytesIO(res_aux.content)) as zfile:
                    zfile.extractall(path)
                break
            except:
                time.sleep(1)
                current_tries += 1

        # convert filenames to lowercase      
        for file in os.listdir(path):
            os.rename(path + file, path + file.lower())
        
        time.sleep(1)

In [6]:
get_fars_data(2010, 2020)